In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('spam.csv', encoding = 'latin-1')
df.head

<bound method NDFrame.head of         v1                                                 v2 Unnamed: 2  \
0      ham  Go until jurong point, crazy.. Available only ...        NaN   
1      ham                      Ok lar... Joking wif u oni...        NaN   
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3      ham  U dun say so early hor... U c already then say...        NaN   
4      ham  Nah I don't think he goes to usf, he lives aro...        NaN   
...    ...                                                ...        ...   
5567  spam  This is the 2nd time we have tried 2 contact u...        NaN   
5568   ham              Will Ì_ b going to esplanade fr home?        NaN   
5569   ham  Pity, * was in mood for that. So...any other s...        NaN   
5570   ham  The guy did some bitching but I acted like i'd...        NaN   
5571   ham                         Rofl. Its true to its name        NaN   

     Unnamed: 3 Unnamed: 4  
0           NaN        NaN  

In [4]:
category = pd.get_dummies(df.v1)
category

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
5567,0,1
5568,1,0
5569,1,0
5570,1,0


In [5]:
new_df = pd.concat([df,category], axis=1)
new_df = new_df.drop(columns='v1')
new_df.head

<bound method NDFrame.head of                                                      v2 Unnamed: 2 Unnamed: 3  \
0     Go until jurong point, crazy.. Available only ...        NaN        NaN   
1                         Ok lar... Joking wif u oni...        NaN        NaN   
2     Free entry in 2 a wkly comp to win FA Cup fina...        NaN        NaN   
3     U dun say so early hor... U c already then say...        NaN        NaN   
4     Nah I don't think he goes to usf, he lives aro...        NaN        NaN   
...                                                 ...        ...        ...   
5567  This is the 2nd time we have tried 2 contact u...        NaN        NaN   
5568              Will Ì_ b going to esplanade fr home?        NaN        NaN   
5569  Pity, * was in mood for that. So...any other s...        NaN        NaN   
5570  The guy did some bitching but I acted like i'd...        NaN        NaN   
5571                         Rofl. Its true to its name        NaN        NaN  

In [6]:
input = new_df['v2'].values
label = new_df[['ham', 'spam']].values

In [7]:
input

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [8]:
label

array([[1, 0],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [9]:
from sklearn.model_selection import train_test_split

kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(input, label, test_size=0.2)

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

token = Tokenizer(num_words = 5000, oov_token = '_')

token.fit_on_texts(kalimat_latih)

sequence_latih = token.texts_to_sequences(kalimat_latih)
sequence_test = token.texts_to_sequences(kalimat_test)

padded_latih = pad_sequences(sequence_latih)
padded_test = pad_sequences(sequence_test)

In [16]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = 5000, output_dim = 16),
    tf.keras.layers.LSTM(32, dropout = 0.5),
    tf.keras.layers.Dense(8, activation = 'relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(2, activation = 'softmax')
])

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'Adam',
    metrics = ['accuracy']
)

In [20]:
class MyCallBack(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nAkurasi sudah melebihi 90%")
      self.model.stop_training = True
callbacks = MyCallBack()

In [21]:
history = model.fit(
    padded_latih, y_latih,
    epochs = 25,
    validation_data = (padded_test,y_test),
    callbacks = [callbacks]
)

Epoch 1/25
140/140 [==============================] - ETA: 0s - loss: 0.0232 - accuracy: 0.9960
Akurasi sudah melebihi 90%
140/140 [==============================] - 12s 88ms/step - loss: 0.0232 - accuracy: 0.9960 - val_loss: 0.0465 - val_accuracy: 0.9883
